In [ ]:
pip install xgboost

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

In [2]:
features = ['tarif_u-v ii. 1 (+ii 2)', 'tarif_u-k (musiker)', 
'tarif_u-k (musiker) mindestverguetung','tarif_u-st i. (musiker)', 
'tarif_e-p', 'tarif_u-k i (+ ii 2b) - mit sonstigem geldwerten vorteil', 
'tarif_u-v iii. 1', 'tarif_u-st i (musiker) nl', 'tarif_u-v iii. 2', 
'tarif_e (musiker)', 
            
#'tarif_u-k iii. 2d) (musiker) - vor geladenen gaesten', 
#'tarif_p-k i. (u-musik)', 'tarif_chorverband konzert u-musik', 
#'tarif_u-v vi. b','tarif_variete i (musiker)',
#'tarif_u-k ii. (musiker) bis 20 min','tarif_u-k ii. (musiker) bis 25 min',
#'tarif_u-k ii. (musiker) bis 15 min','tarif_u-k ii. (musiker) bis 50 min',
#'tarif_u-k ii. (musiker) bis 30 min','tarif_u-k ii. (musiker) bis 10 min',
#'tarif_vk i 3 zirkusunternehmen (musiker)','tarif_p-k i. (e-musik)',
#'tarif_u-st (musiker)','tarif_u-k ii. (musiker) bis 5 min',
#'tarif_u-k ii. (musiker) bis 35 min','tarif_u-k ii. (musiker) bis 40 min',
#'tarif_u-v-ks (musiker)','tarif_u-k ii (musiker) mindestverguetung',
#'tarif_u-k ii. (musiker) bis 45 min',

'place_kirche',	'place_hotel', 'place_theater',	
'place_cafe', 'place_club', 'place_halle',	
'place_schloss', 'place_gaststaette', 'place_festhalle',	
'place_festzelt', 
            
#'place_kulturzentrum', 'place_musikschule',	
#'place_stadthalle', 'place_pub', 'place_gasthof',	
#'place_park', 'place_buergerhaus', 'place_gymnasium',	
#'place_kabarett', 'place_turnhalle', 'place_rathaus',	
#'place_schule', 'place_museum', 'place_arena',	
#'place_schlachthof', 'place_kurhaus', 'place_zelt',	
#'place_grundschule', 'place_forum', 'place_wandelhalle',

# 'location_berlin','location_muenchen','location_hamburg','location_koeln',
# 'location_stuttgart','location_leipzig','location_frankfurt am main',
# 'location_dresden','location_hannover','location_nuernberg','location_duesseldorf',
# 'location_bremen','location_karlsruhe','location_dortmund','location_essen',
# 'location_augsburg','location_mannheim','location_muenster','location_bonn',
# 'location_wiesbaden','location_erfurt','location_kiel','location_regensburg',
# 'location_freiburg im breisgau','location_chemnitz','location_mainz',
# 'location_wuerzburg','location_magdeburg','location_potsdam',
# 'location_bielefeld',

'state_bavaria','state_rhineland-palatinate',
'state_baden-wuerttemberg','state_north rhine-westphalia',	
'state_thuringia','state_hesse',
'state_brandenburg', 'state_schleswig-holstein',
'state_berlin',	'state_mecklenburg-western pomerania',
'state_lower saxony', 'state_hamburg',
'state_saarland', 'state_saxony-anhalt',	
'state_saxony',	'state_bremen',

'vg_datum_year','vg_datum_month','vg_datum_day_of_week','vg_datum_season'
]

In [ ]:
# df = pd.read_pickle('C:\Saravana\Projects\Intellizenz\intellizenz-model-training\data\export_features_2016_2020_v1.pkl.bz2')

In [3]:
cd /home/jupyter/intellizenz-model-training/

/home/jupyter/intellizenz-model-training


In [4]:
df = pd.read_parquet('data/export_features_2016_2020_v1.parquet.gzip')

Baseline Classifier models to classify event segments

In [5]:
X = df[features]
y = df['veranst_segment']

In [6]:
le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

XGBoost Classifier

In [8]:
xgboost_classifier = xgboost.XGBClassifier()
xgboost_classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [9]:
# save the model to disk
xbg_filename = 'model/2022-08-23/xgb_classifier_model.sav'
pickle.dump(xgboost_classifier, open(xbg_filename, 'wb'))

In [10]:
y_pred = xgboost_classifier.predict(X_test)

Evaluation

In [11]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

xgboost_accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(xgboost_accuracy)

[[200803   1597  66046]
 [ 51984   5159  56729]
 [ 80487   3330 156929]]
              precision    recall  f1-score   support

           0       0.60      0.75      0.67    268446
           1       0.51      0.05      0.08    113872
           2       0.56      0.65      0.60    240746

    accuracy                           0.58    623064
   macro avg       0.56      0.48      0.45    623064
weighted avg       0.57      0.58      0.54    623064

362891


SVC Classifier

In [8]:
ls

00-Data-Exploration.ipynb                            README.md
01-prepare-data.ipynb                                baseline/
02-generate-features.ipynb                           data/
03-model-generation-3-step-model-all-features.ipynb  models/
03-model-generation-3-step-model-all-models.ipynb    pyproject.toml
03-model-generation.ipynb                            xgb_classifier_model.sav


In [ ]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

In [ ]:
# save the model to disk
filename = 'model/svc_classifier_model.sav'
pickle.dump(svclassifier, open(filename, 'wb'))

In [ ]:
y_pred = svclassifier.predict(X_test)

Evaluation

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

svc_accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(svc_accuracy)